# RAG Experimentation Notebook

This notebook demonstrates how to query your ingested documents using the RAG pipeline.

**Prerequisites:** Documents must be ingested via the API first.

## Setup: Configure Local Paths and Initialize

In [1]:
import os
import sys
from pathlib import Path

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Set environment variables for LOCAL paths (not Docker paths)
os.environ["DOCUMENTS_PATH"] = str(project_root / "data" / "documents")
os.environ["CHROMA_DB_PATH"] = str(project_root / "data" / "chroma_db")

print(f"✓ Project root: {project_root}")
print(f"✓ Documents path: {os.environ['DOCUMENTS_PATH']}")
print(f"✓ ChromaDB path: {os.environ['CHROMA_DB_PATH']}")

✓ Project root: /Users/kevinknox/coding/agent-api
✓ Documents path: /Users/kevinknox/coding/agent-api/data/documents
✓ ChromaDB path: /Users/kevinknox/coding/agent-api/data/chroma_db


In [12]:
# Import infrastructure components
from exim_agent.config import settings
from exim_agent.infrastructure.db.chroma_client import chroma_client
from exim_agent.infrastructure.llm_providers.openai_provider import initialize_llm_providers

# Initialize LLM and ChromaDB
initialize_llm_providers()
chroma_client.initialize()

# Verify connection
stats = chroma_client.get_collection_stats()
print(f"\n✓ Connected to ChromaDB")
print(f"✓ Collection: {stats['collection_name']}")
print(f"✓ Document count: {stats['document_count']}")
print(f"✓ Status: {stats['status']}")

2025-10-04 13:12:15.284 | INFO     | exim_agent.infrastructure.llm_providers.openai_provider:initialize_llm_providers:12 - Initializing OpenAI LLM and embedding models
2025-10-04 13:12:15.286 | INFO     | exim_agent.infrastructure.llm_providers.openai_provider:initialize_llm_providers:31 - LLM initialized: gpt-5-nano-2025-08-07
2025-10-04 13:12:15.287 | INFO     | exim_agent.infrastructure.llm_providers.openai_provider:initialize_llm_providers:32 - Embedding model initialized: text-embedding-3-small
2025-10-04 13:12:15.287 | INFO     | exim_agent.infrastructure.db.chroma_client:initialize:22 - Initializing ChromaDB at /Users/kevinknox/coding/agent-api/data/chroma_db
2025-10-04 13:12:15.291 | INFO     | exim_agent.infrastructure.db.chroma_client:initialize:42 - ChromaDB initialized successfully with collection: documents



✓ Connected to ChromaDB
✓ Collection: documents
✓ Document count: 2134
✓ Status: active


## Create Query Engine from Existing Vector Store

In [3]:
from llama_index.core import VectorStoreIndex

# Load index from existing ChromaDB collection
index = VectorStoreIndex.from_vector_store(
    chroma_client.get_vector_store()
)

# Create query engine
query_engine = index.as_query_engine(
    similarity_top_k=3,  # Return top 3 most relevant chunks
    response_mode="compact"  # Compact response mode
)

print("✓ Query engine created")

✓ Query engine created


## Test Basic Query

In [4]:
# Test query
query = "What are the key provisions of AGOA?"
response = query_engine.query(query)

print(f"Query: {query}")
print(f"\nResponse:\n{response}")

Query: What are the key provisions of AGOA?

Response:
- Duty-free treatment: AGOA provides duty-free treatment for eligible articles from designated sub-Saharan African beneficiary countries (the program can also include GSP benefits).

- Annual eligibility review and designation: The President conducts an annual review to determine whether countries should continue to be eligible and to decide which additional sub-Saharan African countries, if any, should be designated as beneficiaries.

- Grounds for termination: If a beneficiary country does not make continual progress in meeting eligibility requirements, the President must terminate its designation.

- Targeted withdrawal/suspension: The President may withdraw, suspend, or limit the duty-free treatment for specific articles from a country if that approach would be more effective in promoting compliance than terminating the designation.

- Public process for designations: The designation process involves the AGOA Trade Policy Staff

## Inspect Retrieved Documents

In [7]:
# Get retriever to see what documents are being pulled
retriever = index.as_retriever(similarity_top_k=5)

# Retrieve relevant nodes
query = "What is AfCFTA?"
nodes = retriever.retrieve(query)

print(f"Query: {query}")
print(f"\nRetrieved {len(nodes)} documents:\n")

for i, node in enumerate(nodes, 1):
    print(f"--- Document {i} (Score: {node.score:.3f}) ---")
    print(f"Text preview: {node.text[:200]}...")
    print(f"Metadata: {node.metadata}")
    print()

Query: What is AfCFTA?

Retrieved 3 documents:

--- Document 1 (Score: 0.329) ---
Text preview: Goods Trade between the United States and sub-
Saharan Africa 
 
Billions of Dollars 
 
 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 
Total...
Metadata: {'page_label': '1', 'file_name': 'US Trade with sub_Saharan Africa 11162023_0.pdf', 'file_path': '/app/data/documents/pdf/US Trade with sub_Saharan Africa 11162023_0.pdf', 'file_type': 'application/pdf', 'file_size': 97926, 'creation_date': '2025-10-03', 'last_modified_date': '2025-10-03'}

--- Document 2 (Score: 0.312) ---
Text preview: U.S. Imports under the African Growth and Opportunity Act (AGOA)  
AGOA (including GSP) imports for 2022 totaled $10.3 billion, up 26 percent compared to 2001 (the first 
full-year of AGOA trade) and ...
Metadata: {'page_label': '1', 'file_name': 'AGOA Trade Fact Sheet_final.pdf', 'file_path': '/app/data/documents/pdf/AGOA Trade Fact Sheet_final.pd

## Experiment with Different Response Modes

### 1. "compact" mode is one shot dense retrieval thats best for short queries and simple questions
### 2. "tree_summarize" mode is for hierarchical reasoning and complex questions

In [9]:
# Compare different response modes
query = "Explain incoterms and their importance in international trade"

# Compact mode (default)
engine_compact = index.as_query_engine(
    similarity_top_k=3,
    response_mode="compact"
)

# Tree summarize mode
engine_tree = index.as_query_engine(
    similarity_top_k=5,
    response_mode="tree_summarize"
)

response_compact = engine_compact.query(query)
response_tree = engine_tree.query(query)

print("=== COMPACT MODE ===")
print(response_compact)
print("\n=== TREE SUMMARIZE MODE ===")
print(response_tree)

=== COMPACT MODE ===
- What they are: Incoterms are internationally accepted selling terms that the buyer and seller agree on. They define which tasks, costs, and risks are borne by each party and indicate when those costs and risks transfer from seller to buyer.

- Who publishes them: The terms are published by the International Chamber of Commerce (ICC) and are tied to international commercial law. They are widely accepted by governments and legal authorities around the world.

- Edition and updates: The ICC updates Incoterms periodically to reflect changes in global trade. The 2020 edition became effective on January 1, 2020. The terms have evolved since the first publication in 1936.

- How they are used: Contracts specify an Incoterm (and the edition year). Incoterms come with guidance on the obligations and charges for the seller and the buyer, helping to clarify who does what (loading, shipping, insurance, customs, etc.) and who bears which costs.

- Why they are important in in

## Test Different Top-K Values
### Responses are based on the aggregation of different top-k values

In [14]:
# Test how many documents to retrieve
query = "What are the customs tariff schedules for South Africa?"

for k in [1, 3, 5]:
    engine = index.as_query_engine(similarity_top_k=k)
    response = engine.query(query)
    
    print(f"\n=== Top-K = {k} ===")
    print(f"Response length: {len(str(response))} chars")
    print(f"Response: {str(response)[:200]}...")


=== Top-K = 1 ===
Response length: 497 chars
Response: The schedules shown are from Schedule 1, Part 1, Section XI of the South Africa Customs & Excise Tariff. For the HS subheadings listed:

- 6006.44.10 Printed: Tulle
  - General: 5%
  - EU/UK: 3%
  - E...

=== Top-K = 3 ===
Response length: 589 chars
Response: The schedules shown are parts of South Africa’s Customs & Excise Tariff (Schedule 1, Part 1). They include product codes and region-specific duty rates (General, EU/UK, EFTA, SADC, MERCOSUR, AfCFTA). ...

=== Top-K = 5 ===
Response length: 760 chars
Response: The customs tariff schedules are in Schedule 1, Part 1, for sections II and XI of the SA Customs & Excise Tariff. They list commodity codes (HS codes), units, and the duty rates by trading-region grou...


## Experiment with LLM Temperature

In [18]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAIResponses

query = "Summarize the benefits of the African Continental Free Trade Area"

# Low temperature (more factual)
Settings.llm = OpenAIResponses(model="gpt-5-nano-2025-08-07", temperature=0.1)
engine_low = index.as_query_engine(similarity_top_k=3)
response_low = engine_low.query(query)

# High temperature (more creative)
Settings.llm = OpenAIResponses(model="gpt-5-nano-2025-08-07", temperature=0.9)
engine_high = index.as_query_engine(similarity_top_k=3)
response_high = engine_high.query(query)

print("=== LOW TEMPERATURE (0.1) - Factual ===")
print(f"Response length: {len(str(response_low))} chars")
print(response_low)
print("\n=== HIGH TEMPERATURE (0.9) - Creative ===")
print(f"Response length: {len(str(response_high))} chars")
print(response_high)

# Reset to default
Settings.llm = OpenAIResponses(model="gpt-5-nano-2025-08-07", temperature=0.7)

=== LOW TEMPERATURE (0.1) - Factual ===
Response length: 975 chars
The African Continental Free Trade Area (AfCFTA) benefits are not described here. The materials focus on AGOA (African Growth and Opportunity Act) trade preferences and U.S.–sub-Saharan Africa trade flows. Key AGOA-related points include:

- AGOA offers duty-free (tariff-preferred) treatment for eligible goods from designated sub-Saharan African beneficiary countries.
- Imports are tracked in categories such as AGOA oil imports, AGOA non-oil imports (including GSP), and AGOA apparel imports.
- For 2025, 32 sub-Saharan African countries were designated as AGOA beneficiaries; several countries were not.
- Eligibility is reviewed annually, and the President may terminate or modify a country’s designation or adjust duty-free treatment to promote compliance.
- There is a formal process for comments and a public hearing as part of the annual eligibility review.

If you’d like, I can provide a concise summary focused specifica

## Test Multiple Queries

In [19]:
# Test various queries on your documents
test_queries = [
    "What are the benefits of AGOA for African countries?",
    "Explain the dual-use goods and technologies list",
    "What is PVoC and how does it apply to Kenya?",
    "Summarize US trade with sub-Saharan Africa",
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    response = query_engine.query(query)
    print(f"\n{response}\n")


Query: What are the benefits of AGOA for African countries?

- Duty-free access to the U.S. market for eligible products from AGOA-eligible sub-Saharan African countries (including goods covered under AGOA and GSP).

- This has supported substantial trade with the United States, with AGOA imports totaling about $10.3 billion in 2022 (and non-oil AGOA imports of $5.7 billion), indicating strong market access and growth potential.

- It has encouraged diversification into non-oil sectors, with notable gains in apparel, auto parts, macadamia nuts, jewelry, fresh oranges, and footwear.

- The program covers key import categories such as Mineral Fuels, Transportation Equipment, Textiles and Apparel, Agricultural Products, Minerals/Metals, and Jewelry.

- A broad group of countries benefits from AGOA, with 32 sub-Saharan African countries designated as beneficiaries for 2025.


Query: Explain the dual-use goods and technologies list

- What it is: A list of goods and technologies that have 

## Evaluate Retrieval Quality

In [20]:
# Analyze retrieval scores
test_queries = [
    "What are incoterms?",
    "Explain AGOA eligibility criteria",
    "What is the AfCFTA agreement?"
]

retriever = index.as_retriever(similarity_top_k=5)

for query in test_queries:
    nodes = retriever.retrieve(query)
    scores = [node.score for node in nodes]
    
    print(f"\nQuery: {query}")
    print(f"Retrieved: {len(nodes)} documents")
    print(f"Scores: {[f'{s:.3f}' for s in scores]}")
    print(f"Avg score: {sum(scores)/len(scores):.3f}")
    print(f"Top score: {max(scores):.3f}")


Query: What are incoterms?
Retrieved: 3 documents
Scores: ['0.596', '0.500', '0.485']
Avg score: 0.527
Top score: 0.596

Query: Explain AGOA eligibility criteria
Retrieved: 3 documents
Scores: ['0.507', '0.501', '0.433']
Avg score: 0.480
Top score: 0.507

Query: What is the AfCFTA agreement?
Retrieved: 3 documents
Scores: ['0.330', '0.310', '0.304']
Avg score: 0.315
Top score: 0.330


## Advanced: Chat Engine with Memory

### Chat modes:
#### ChatMode.BEST (default): Chat engine that uses an agent (react or openai) with a query engine tool
#### ChatMode.CONTEXT: Chat engine that uses a retriever to get context
#### ChatMode.CONDENSE_QUESTION: Chat engine that condenses questions
#### ChatMode.CONDENSE_PLUS_CONTEXT: Chat engine that condenses questions and uses a retriever to get context
#### ChatMode.SIMPLE: Simple chat engine that uses the LLM directly
#### ChatMode.REACT: Chat engine that uses a react agent with a query engine tool
#### ChatMode.OPENAI: Chat engine that uses an openai agent with a query engine tool

In [21]:
# Create a chat engine that maintains conversation history
chat_engine = index.as_chat_engine(
    chat_mode="context",
    similarity_top_k=3
)

# Have a conversation
response1 = chat_engine.chat("What is AGOA?")
print("User: What is AGOA?")
print(f"Assistant: {response1}\n")

response2 = chat_engine.chat("What are its main benefits?")
print("User: What are its main benefits?")
print(f"Assistant: {response2}\n")

response3 = chat_engine.chat("Which countries are eligible?")
print("User: Which countries are eligible?")
print(f"Assistant: {response3}")

User: What is AGOA?
Assistant: AGOA stands for the African Growth and Opportunity Act. It is a U.S. trade statute designed to promote economic growth in sub-Saharan Africa by giving eligible African countries preferential access to the U.S. market.

Key points:
- It provides duty-free or reduced-duty treatment for a wide range of products from eligible countries (often combined with the Generalized System of Preferences, or GSP, benefits).
- Countries are designated as AGOA beneficiaries; the list is reviewed and updated annually (for 2025, 32 sub-Saharan African countries were designated).
- The President can terminate eligibility or suspend/withdraw duty-free treatment for specific goods if a country does not meet the requirements.
- An AGOA subcommittee (AGOA TPSC Subcommittee) handles the annual eligibility review and related procedures.

User: What are its main benefits?
Assistant: Here are the main benefits of AGOA (the African Growth and Opportunity Act):

- Tariff preferences: 

## Next Steps

After experimenting here, you can:

1. **Implement Chat Service** - Move best parameters to `application/chat_service/`
2. **Add Evaluation Metrics** - Implement in `application/evaluation_service/`
3. **Create Custom Prompts** - Store in `domain/prompts/`
4. **Build Tools** - Add custom tools in `domain/tools/`
5. **Optimize Performance** - Tune chunk size, overlap, top-k values